In [108]:
import csv
import numpy as np
import pandas as pd
import utils 

In [40]:
data = np.genfromtxt('repeated_data.csv', delimiter=',', dtype=str)
data = np.delete(data, 0, 0)

In [42]:
data.shape

(28, 17)

In [84]:
new_data = []

# turn each game into entry in this array

def remove_blank(l):
    return list(filter(lambda a: a != '', l))

def replace_letters(l):
   return [0 if x=='c' else 1 for x in l]

for i in range(0, data.shape[0], 2):
    p1 = data[i]
    p2 = data[i+1]

    game = int(p1[0])
    player1 = p1[2]
    player2 = p2[2]
    p1_actions = replace_letters(remove_blank(p1[3:]))
    p2_actions = replace_letters(remove_blank(p2[3:]))
    actions = np.vstack((p1_actions, p2_actions))

    new_data.append([game, player1, player2, actions])


In [85]:
df = pd.DataFrame(new_data)
df.columns = ['Game', 'Player_1', 'Player_2', 'Actions']

In [86]:
df.head()

,Game,Player_1,Player_2,Actions
0,0,Faisal,Bharathvaj,"[[1, 0, 0, 0, 1], [0, 1, 0, 1, 1]]"
1,1,yuan,Dagmar,"[[0, 1, 1], [0, 0, 1]]"
2,2,Ehsan,Mohammend,"[[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, ..."
3,3,Tao,Rohini,"[[1, 0, 1, 0, 0], [0, 1, 1, 0, 0]]"
4,4,Anita,Greg,"[[0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0]]"


In [91]:
plays = np.zeros((2,2))


for i, row in df.iterrows():
    actions = row.Actions
    for round in range(actions.shape[1]):
        play = actions[:, round]
        outcome = play[0]*2 + play[1]
        row_idx = outcome  // 2
        col_idx = outcome % 2
        plays[row_idx, col_idx] += 1

In [92]:
# frequency of each outcome
plays/np.sum(plays)

array([[0.30232558, 0.18604651],
       [0.26744186, 0.24418605]])

# Model 1: Players (Quantally) Best Respond to a Sub-History

In [106]:
def EU(actions):
    '''
    Given actions by both players returns the sum of payoffs for each player
    '''

    player_0_payoffs = [0, -1, 1, -2] # row payoffs
    player_1_payoffs = [0, 1, -1, -2] # col payoffs

    player_0_sum = 0
    player_1_sum = 0
    for round in actions.T:

        outcome = round[ 0]*2 + round# [1]
        player_0_sum += player_0_payoffs[outcome]
        player_1_sum += player_1_payoffs[outcome]
    
    return player_0_sum, player_1_sum

In [101]:
# this is pretty much quantal regret. I'll read the paper tomorrow and implement this method

def quantal_history_best_response(actions, k):
    '''
    actions : (np.array) both of the player's actions
    k : (int) the length of the subhistory considered 
    '''
    
    def best_response(last_k_moves):
        action_0_moves = np.zeros_like(last_k_moves)
        action_1_moves = np.ones_like(last_k_moves)


    for round in range(actions.shape[1]):
        for player in range(0,2):

            if k > round:
                last_k_moves = actions[1-player, 0:round]
            else:
                last_k_moves = actions[1-player, round-k:round] # last k moves by the opponent

In [107]:

for i, row in df.iterrows():
    actions = row.Actions
    print(actions)
    print(EU(actions))
    #quantal_history_best_response(actions, 2)
    assert False

[[1 0 0 0 1]
 [0 1 0 1 1]]
(-3, -1)


AssertionError: 

In [110]:
payoffs = np.zeros([2, 2, 2])
payoffs[0, :, :] = np.array([[0, -1],
                            [1, -2]])
payoffs[1, :, :] = np.array([[0, 1],
                            [-1, -2]])

In [ ]:
class Level_K_Model:
    def __init__(self, K):
        self.K = K
        self.payoffs = [[0, -1, 1, -2],
                        [0, 1, -1, -2]]     # row payoffs
        self.alphas = np.zeros((K, 1))


    def log_likelihood(self, dataset):
        '''
        Given the dataset and the fitted model, returns the log likelihood 
        
        dataset : list of plays, each play is an np array
        '''


        for play in dataset:




    def fit():

        
        
        










        
        


    def compute_BR(self, i, s_other):
        '''
        i : index of player for whom we are computing BR
        s_other : strategy profile of other player
        '''
    
        # get EU of action 0
        if i == 0:
            s = [np.array([1, 0]), s_other]
        else:
            s = [s_other, np.array([1, 0])]
        eu_0 = utils.expected_utility(s, self.payoffs)

        # get EU of action 1
        if i == 0:
            s = [np.array([0, 1]), s_other]
        else:
            s = [s_other, np.array([0, 1])]
        eu_1 = utils.expected_utility(s, self.payoffs)

        # return action with greater EU
        if eu_0 >= eu_1:
            return np.array([1, 0])
        else:
            return np.array([0, 1])


    def predict(self, history=None):
        predictions = np.zeros((4, self.k)) # first two rows are mixed strategy of first player, second two are of second player
        
        # level-0 base case
        if history == None:
            predictions[0:2, 0] = np.array([0.5, 0.5])
            predictions[2:4, 0] = np.array([0.5, 0.5])
        else:
            predictions[0:2, 0] = np.sum(history[0:2, :], axis=1)/ np.sum(history[0:2, :])
            predictions[2:4, 0] = np.sum(history[2:4, :], axis=1)/ np.sum(history[2:4, :])

        
        for k in range(1, self.K):
            for player in range(0,2):
                # best respond to 1 level lower
                if player == 0:
                    predictions[0:2, k] = self.compute_BR(player,  predictions[2:4, k-1])
                else:
                    predictions[2:4, k] = self.compute_BR(player,  predictions[0:2, k-1])


        # return overall predictions
        p0_prediction = np.matmul(predictions[0:2, :], self.alphas)
        p1_prediction = np.matmul(predictions[2:4, :], self.alphas)

        return p0_prediction, p1_prediction


In [ ]:
# another one: player play as if their opponent is going to best resond to their last move
# generalization : level_0 strategy is some function of the history,  then players either use CH or level-k to produce the strategy for the next
# iteraion